In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,User,Current session?
2,application_1717826852964_0003,pyspark,idle,Link,Link,None,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
%%bash
## this is a bash command
#wget -O - https://redfin-public-data.s3.us-west-2.amazonaws.com/redfin_market_tracker/city_market_tracker.tsv000.gz | aws s3 cp - s3://store-raw-data-redfin-yml/city_market_tracker.tsv000.gz 

In [2]:
spark = SparkSession.builder.appName("RedfinDataanalysis").getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
df = spark.read.csv('s3://store-raw-data-redfin-yml/city_market_tracker.tsv000.gz', header=True, inferSchema=True, sep='\t' )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- period_begin: date (nullable = true)
 |-- period_end: date (nullable = true)
 |-- period_duration: integer (nullable = true)
 |-- region_type: string (nullable = true)
 |-- region_type_id: integer (nullable = true)
 |-- table_id: integer (nullable = true)
 |-- is_seasonally_adjusted: string (nullable = true)
 |-- region: string (nullable = true)
 |-- city: string (nullable = true)
 |-- state: string (nullable = true)
 |-- state_code: string (nullable = true)
 |-- property_type: string (nullable = true)
 |-- property_type_id: integer (nullable = true)
 |-- median_sale_price: double (nullable = true)
 |-- median_sale_price_mom: double (nullable = true)
 |-- median_sale_price_yoy: double (nullable = true)
 |-- median_list_price: double (nullable = true)
 |-- median_list_price_mom: double (nullable = true)
 |-- median_list_price_yoy: double (nullable = true)
 |-- median_ppsf: double (nullable = true)
 |-- median_ppsf_mom: double (nullable = true)
 |-- median_ppsf_yoy: double (nul

In [6]:
df.columns

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['period_begin', 'period_end', 'period_duration', 'region_type', 'region_type_id', 'table_id', 'is_seasonally_adjusted', 'region', 'city', 'state', 'state_code', 'property_type', 'property_type_id', 'median_sale_price', 'median_sale_price_mom', 'median_sale_price_yoy', 'median_list_price', 'median_list_price_mom', 'median_list_price_yoy', 'median_ppsf', 'median_ppsf_mom', 'median_ppsf_yoy', 'median_list_ppsf', 'median_list_ppsf_mom', 'median_list_ppsf_yoy', 'homes_sold', 'homes_sold_mom', 'homes_sold_yoy', 'pending_sales', 'pending_sales_mom', 'pending_sales_yoy', 'new_listings', 'new_listings_mom', 'new_listings_yoy', 'inventory', 'inventory_mom', 'inventory_yoy', 'months_of_supply', 'months_of_supply_mom', 'months_of_supply_yoy', 'median_dom', 'median_dom_mom', 'median_dom_yoy', 'avg_sale_to_list', 'avg_sale_to_list_mom', 'avg_sale_to_list_yoy', 'sold_above_list', 'sold_above_list_mom', 'sold_above_list_yoy', 'price_drops', 'price_drops_mom', 'price_drops_yoy', 'off_market_in_two_wee

In [8]:
df = df.select(['period_end','period_duration', 'city', 'state', 'property_type',
    'median_sale_price', 'median_ppsf', 'homes_sold', 'inventory', 'months_of_supply', 'median_dom', 'sold_above_list', 'last_updated'])
df.show(3)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------+---------------+----------+--------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|period_end|period_duration|      city|   state|       property_type|median_sale_price|       median_ppsf|homes_sold|inventory|months_of_supply|median_dom|    sold_above_list|       last_updated|
+----------+---------------+----------+--------+--------------------+-----------------+------------------+----------+---------+----------------+----------+-------------------+-------------------+
|2020-04-30|             30|Aspen Hill|Maryland|Single Family Res...|         543474.0|249.36921783010933|        27|       40|             1.5|         8|0.48148148148148145|2024-05-19 14:23:50|
|2018-09-30|             30| Riverside|    Iowa|     All Residential|         212550.0|109.68417945690673|         2|       11|             5.5|       595|                0.5|2024-05-19 14:23:50|
|2018-04-30|        

In [10]:
df.count() ## Total number of rows

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5354687

In [20]:
from pyspark.sql.functions import isnull
null_values = [df.where(isnull(col_name)).count() for col_name in df.columns]
null_values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[0, 0, 0, 0, 0, 6138, 69691, 5719, 423284, 344188, 70742, 36724, 0]

In [22]:
for i, col_name in enumerate(df.columns):
    print(f'{col_name}: {null_values[i]}')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

period_end: 0
period_duration: 0
city: 0
state: 0
property_type: 0
median_sale_price: 6138
median_ppsf: 69691
homes_sold: 5719
inventory: 423284
months_of_supply: 344188
median_dom: 70742
sold_above_list: 36724
last_updated: 0

In [23]:
df = df.dropna() ## dropping rows which have any null values

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
df.count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

4842224

In [26]:
from pyspark.sql.functions import year, month

df=df.withColumn('year', year(df.period_end))
df=df.withColumn('month', month(df.period_end))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df = df.drop('period_end', 'last_updated')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
from pyspark.sql.functions import when

#let's map the month number to their respective month name.

df = df.withColumn("period_end_month", 
                   when(col("month") == 1, "January")
                   .when(col("month") == 2, "February")
                   .when(col("month") == 3, "March")
                   .when(col("month") == 4, "April")
                   .when(col("month") == 5, "May")
                   .when(col("month") == 6, "June")
                   .when(col("month") == 7, "July")
                   .when(col("month") == 8, "August")
                   .when(col("month") == 9, "September")
                   .when(col("month") == 10, "October")
                   .when(col("month") == 11, "November")
                   .when(col("month") == 12, "December")
                   .otherwise("Unknown")
                 )

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
s3_bucket = "s3://redfin-transformed-zone/redfin_data.parquet"
df.write.mode("overwrite").parquet(s3_bucket)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…